In [1]:
import numpy as np

In [ ]:
# load data

x_l2 = np.loadtxt('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/train_input_fidelity_0.txt')


# HF input and output
x_h = np.loadtxt('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/train_input_fidelity_1.txt')


In [3]:
# load all z 
z_str = ['0', '0.2', '0.5', '1', '2', '3']

y_l2 = []
y_h = []
for i in range(len(z_str)):
    y_l2.append(np.loadtxt('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z' + z_str[i] + '/train_output_fidelity_0.txt'))
    y_h.append(np.loadtxt('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z' + z_str[i] + '/train_output_fidelity_1.txt'))
y_h = np.concatenate(y_h, axis=1)
y_l2 = np.concatenate(y_l2, axis=1)


In [4]:
y_h.shape

(21, 384)

In [5]:
y_l2.shape

(564, 384)

In [6]:
y_h.shape

(21, 384)

In [7]:
# loop over the L1 input and find the ones (indices) that are in the HF input
indices = []
for i in range(len(x_l2)):
    if x_l2[i] in x_h:
        indices.append(i)


In [8]:
indices

[24,
 25,
 26,
 54,
 55,
 56,
 72,
 73,
 74,
 207,
 208,
 209,
 240,
 241,
 242,
 300,
 301,
 302,
 522,
 523,
 524]

In [9]:
# compute the ratio
r_h = 10**y_h / 10**y_l2[indices]

In [12]:
xy_l2 = np.concatenate((x_l2[indices], y_l2[indices]), axis=1)

In [15]:
xy_l2.shape

(21, 394)

In [19]:
# save the dataset
save_dir_L = 'muse_L2'
save_dir_LHr = 'muse_L2Hr'
save_dir_LH = 'muse_L2H'

# mkdir -p the dirs
import os
os.makedirs(save_dir_L, exist_ok=True)
os.makedirs(save_dir_LHr, exist_ok=True)
os.makedirs(save_dir_LH, exist_ok=True)

np.savetxt(save_dir_L + '/train_input.txt', x_l2)
np.savetxt(save_dir_L + '/train_output.txt', y_l2)

np.savetxt(save_dir_LHr + '/train_input.txt', x_h)
np.savetxt(save_dir_LHr + '/train_output.txt', r_h)

np.savetxt(save_dir_LH + '/train_input.txt', xy_l2)
np.savetxt(save_dir_LH + '/train_output.txt', y_h)

# copy kf.txt
import shutil
shutil.copyfile('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/kf.txt', save_dir_L + '/kf.txt')
shutil.copyfile('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/kf.txt', save_dir_LHr + '/kf.txt')
shutil.copyfile('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/kf.txt', save_dir_LH + '/kf.txt')


'muse_L2H/kf.txt'

In [21]:
from sklearn.decomposition import PCA

def pca_decomp(y, explained_min=0.999, standardize=False):
        pca = PCA()
        pca.fit(y)
        # Explained variance ratio for each PC
        explained_variance = pca.explained_variance_ratio_

        # Cumulative explained variance
        cumulative_variance = np.cumsum(explained_variance)

        # Find the number of PCs that explain args.min_pca of the variance
        n_components = np.argmax(cumulative_variance > explained_min) + 1

        # Transform the data, only keep the first n_components
        y = pca.transform(y)[:, :n_components]

        # standardize the output data (PCA coefficients) (not recommended)
        if standardize:
            y_mean = np.mean(y, axis=0)
            y_std = np.std(y, axis=0)
            y = (y - y_mean) / y_std
        else:
            y_mean = None
            y_std = None

        # save the PCA components
        mean_std = {'pca_components': pca.components_[:n_components], 'pca_mean': pca.mean_, 'mean': y_mean, 'std': y_std}

        # print the number of PCs and the explained variance
        print(f"Number of PCs: {n_components}")
        print(f"Explained variance: {cumulative_variance[n_components - 1]}")

        return y, mean_std

# PCA global
y_l2_pca_g, mean_std_g = pca_decomp(y_l2, explained_min=.99999)

Number of PCs: 21
Explained variance: 0.999991656153113


In [22]:
xy_l2_pca_g = np.concatenate((x_l2[indices], y_l2_pca_g[indices]), axis=1)

In [23]:
xy_l2_pca_g.shape

(21, 31)

In [24]:
# save L1H_pca_g
save_dir_LH_pca_g = 'muse_L2H_pca_g'
os.makedirs(save_dir_LH_pca_g, exist_ok=True)
np.savetxt(save_dir_LH_pca_g + '/train_input.txt', xy_l2_pca_g)
np.savetxt(save_dir_LH_pca_g + '/train_output.txt', y_h)

shutil.copyfile('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/kf.txt', save_dir_LH_pca_g + '/kf.txt')



'muse_L2H_pca_g/kf.txt'

In [25]:
# pca local
n_pc_zs = []
y_pca = []
n_z = len(z_str)
n_k = len(np.loadtxt('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/kf.txt'))
# y = y.reshape(-1, n_z, n_k)
for i in range(n_z):
    ik_start = i * n_k
    ik_end = (i + 1) * n_k
    y_pca_z, mean_std_z = pca_decomp(y_l2[:, ik_start:ik_end], explained_min=.99999)
    # n_pc_zs.append(y_pca_z.shape[1])
    y_pca.append(y_pca_z)

# concatenate the PCA coefficients
y_l2_pca_l = np.concatenate(y_pca, axis=1)

Number of PCs: 10
Explained variance: 0.9999923957378889
Number of PCs: 9
Explained variance: 0.9999909390884748
Number of PCs: 9
Explained variance: 0.99999263128643
Number of PCs: 8
Explained variance: 0.9999919136131744
Number of PCs: 7
Explained variance: 0.9999903121700564
Number of PCs: 7
Explained variance: 0.9999919788750318


In [26]:
y_l2_pca_l.shape

(564, 50)

In [27]:
xy_l2_pca_l = np.concatenate((x_l2[indices], y_l2_pca_l[indices]), axis=1)

In [28]:
# save L1H_pca_l
save_dir_LH_pca_l = 'muse_L2H_pca_l'
os.makedirs(save_dir_LH_pca_l, exist_ok=True)
np.savetxt(save_dir_LH_pca_l + '/train_input.txt', xy_l2_pca_l)
np.savetxt(save_dir_LH_pca_l + '/train_output.txt', y_h)

shutil.copyfile('./wide/matter_power_564_Box250_Part750_21_Box1000_Part3000_z0/kf.txt', save_dir_LH_pca_l + '/kf.txt')



'muse_L2H_pca_l/kf.txt'